In [10]:
import pandas as pd
import numpy as np
import itertools
import csv

# Configuration
target_years_int = list(range(1989, 2015))

# Mappings (Polish -> ISO)
map_pol_to_iso = {
    'Polska': 'POL', 'Czechy': 'CZE', 'Słowacja': 'SVK', 'Węgry': 'HUN',
    'Słowenia': 'SVN', 'Estonia': 'EST', 'Łotwa': 'LVA', 'Litwa': 'LTU',
    'Rumunia': 'ROU', 'Bułgaria': 'BGR', 'Rosja': 'RUS', 'Ukraina': 'UKR',
    'Białoruś': 'BLR', 'Kazachstan': 'KAZ'
}
map_iso_to_pol = {v: k for k, v in map_pol_to_iso.items()}
target_iso_codes = list(map_pol_to_iso.values())

# Transition Mappings
map_trans_name_to_iso = {
    'POLAND': 'POL', 'CZECH REPUBLIC': 'CZE', 'SLOVAK REPUBLIC': 'SVK',
    'HUNGARY': 'HUN', 'SLOVENIA': 'SVN', 'ESTONIA': 'EST',
    'LATVIA': 'LVA', 'LITHUANIA': 'LTU', 'ROMANIA': 'ROU',
    'BULGARIA': 'BGR', 'RUSSIAN FEDERATION': 'RUS', 'UKRAINE': 'UKR',
    'BELARUS': 'BLR', 'KAZAKHSTAN': 'KAZ'
}
target_trans_countries = list(map_trans_name_to_iso.keys())

# V-Dem Mappings
map_vdem_to_iso = {
    'Poland': 'POL', 'Czechia': 'CZE', 'Slovakia': 'SVK', 'Hungary': 'HUN',
    'Slovenia': 'SVN', 'Estonia': 'EST', 'Latvia': 'LVA', 'Lithuania': 'LTU',
    'Romania': 'ROU', 'Bulgaria': 'BGR', 'Russia': 'RUS', 'Ukraine': 'UKR',
    'Belarus': 'BLR', 'Kazakhstan': 'KAZ'
}
target_vdem_countries = list(map_vdem_to_iso.keys())

# Employment Mappings
map_empl_to_iso = {
    'Poland': 'POL', 'Czech Republic': 'CZE', 'Slovak Republic': 'SVK', 'Slovakia': 'SVK',
    'Hungary': 'HUN', 'Slovenia': 'SVN', 'Estonia': 'EST', 'Latvia': 'LVA',
    'Lithuania': 'LTU', 'Romania': 'ROU', 'Bulgaria': 'BGR', 'Russian Federation': 'RUS',
    'Ukraine': 'UKR', 'Belarus': 'BLR', 'Kazakhstan': 'KAZ'
}


# --- 1. Transition Indicators ---
trans_csv = 'Transition-indicators-by-country (1).csv'
df_trans = pd.read_csv(trans_csv, sep=';', decimal=',')
df_trans.rename(columns={df_trans.columns[0]: 'Country', df_trans.columns[1]: 'Indicator'}, inplace=True)
df_trans['Country'] = df_trans['Country'].ffill()
df_trans = df_trans[df_trans['Country'].isin(target_trans_countries)]

cols_to_keep = ['Country', 'Indicator'] + [str(y) for y in target_years_int if str(y) in df_trans.columns]
df_trans = df_trans[cols_to_keep]

df_trans_melt = df_trans.melt(id_vars=['Country', 'Indicator'], var_name='Year', value_name='Value')
df_trans_melt['Year'] = df_trans_melt['Year'].astype(int)
df_trans_pivot = df_trans_melt.pivot_table(index=['Country', 'Year'], columns='Indicator', values='Value', aggfunc='first').reset_index()
df_trans_pivot['Country_Code'] = df_trans_pivot['Country'].map(map_trans_name_to_iso)
df_trans_pivot.drop(columns=['Country'], inplace=True)

# --- 2. Master Skeleton ---
skeleton = pd.DataFrame(list(itertools.product(target_iso_codes, target_years_int)), columns=['Country_Code', 'Year'])
df_final = pd.merge(skeleton, df_trans_pivot, on=['Country_Code', 'Year'], how='left')

# --- 3. SDR Files ---
files_map_sdr = {
    'SDR_malignant neoplasms.csv': 'SDR_Malignant_Neoplasms',
    'SDR_external causes of injury and poisoning.csv': 'SDR_External_Causes',
    'SDR_Selected alcohol-related causes.csv': 'SDR_Alcohol_Related',
    'SDR_Suicide and self-inflicted injury.csv': 'SDR_Suicide',
    'SDR_Ischaemic heart disease.csv': 'SDR_Ischaemic_Heart',
    'SDR_diseases of circulatory system.csv': 'SDR_Circulatory_System',
    'Life expectancy at birth (years).csv': 'Life_Expectancy'
}
for fname, col_name in files_map_sdr.items():
    try:
        df_temp = pd.read_csv(fname)
        if 'SEX' in df_temp.columns:
            df_temp = df_temp[df_temp['SEX'] == 'ALL']
        df_temp = df_temp[df_temp['COUNTRY'].isin(target_iso_codes)]
        df_temp = df_temp[df_temp['YEAR'].isin(target_years_int)]
        df_temp = df_temp[['COUNTRY', 'YEAR', 'VALUE']].copy()
        df_temp.rename(columns={'COUNTRY': 'Country_Code', 'YEAR': 'Year', 'VALUE': col_name}, inplace=True)
        df_final = pd.merge(df_final, df_temp, on=['Country_Code', 'Year'], how='left')
    except Exception as e:
        print(f"Skipping {fname}: {e}")

# --- 4. V-Dem ---
vdem_file = 'v_dem.csv'
df_vdem = pd.read_csv(vdem_file, sep=';', decimal='.')
df_vdem = df_vdem[df_vdem['country_name'].isin(target_vdem_countries)]
df_vdem = df_vdem[df_vdem['year'].isin(target_years_int)]
df_vdem['Country_Code'] = df_vdem['country_name'].map(map_vdem_to_iso)
df_vdem = df_vdem[['Country_Code', 'year', 'v2x_cspart', 'v2pehealth']]
df_vdem.rename(columns={'year': 'Year', 'v2x_cspart': 'Civil_Society_Participation', 'v2pehealth': 'Health_Equality'}, inplace=True)
df_final = pd.merge(df_final, df_vdem, on=['Country_Code', 'Year'], how='left')

# --- 5. GDP per Capita ---
gdp_file = 'gdp_per_capita.csv'
df_gdp = pd.read_csv(gdp_file, sep=';')
df_gdp = df_gdp[df_gdp['countrycode'].isin(target_iso_codes)]
df_gdp = df_gdp[df_gdp['year'].isin(target_years_int)]
def clean_gdp(x):
    if isinstance(x, str):
        return x.replace(' ', '')
    return x
df_gdp['gdppc'] = df_gdp['gdppc'].apply(clean_gdp)
df_gdp['gdppc'] = pd.to_numeric(df_gdp['gdppc'], errors='coerce')
df_gdp = df_gdp[['countrycode', 'year', 'gdppc']].copy()
df_gdp.rename(columns={'countrycode': 'Country_Code', 'year': 'Year', 'gdppc': 'GDP_Per_Capita'}, inplace=True)
df_final = pd.merge(df_final, df_gdp, on=['Country_Code', 'Year'], how='left')

# --- 6. Alcohol Consumption ---
alc_file = 'alcohol_consumption.csv'
df_alc = pd.read_csv(alc_file, sep=';', decimal='.')
df_alc = df_alc[df_alc['COUNTRY'].isin(target_iso_codes)]
df_alc = df_alc[df_alc['TIME_PERIOD'].isin(target_years_int)]
df_alc = df_alc[['COUNTRY', 'TIME_PERIOD', 'ALCOHOL_CONSUPTION_LITER_PER_PERSON']].copy()
df_alc.rename(columns={'COUNTRY': 'Country_Code', 'TIME_PERIOD': 'Year', 'ALCOHOL_CONSUPTION_LITER_PER_PERSON': 'Alcohol_Consumption'}, inplace=True)
df_final = pd.merge(df_final, df_alc, on=['Country_Code', 'Year'], how='left')

# --- 7. War Dummy ---
df_final['War'] = 0
df_final.loc[(df_final['Country_Code'] == 'ROU') & (df_final['Year'] == 1989), 'War'] = 1
df_final.loc[(df_final['Country_Code'] == 'SVN') & (df_final['Year'] == 1991), 'War'] = 1
russia_war_years = [1994, 1995, 1996] + list(range(1999, 2010)) + [2014]
df_final.loc[(df_final['Country_Code'] == 'RUS') & (df_final['Year'].isin(russia_war_years)), 'War'] = 1
df_final.loc[(df_final['Country_Code'] == 'UKR') & (df_final['Year'] == 2014), 'War'] = 1

# --- 8. Government Expenditure ---
gov_file = 'government_final_consumption_expenditure.csv'
df_gov = pd.read_csv(gov_file, sep=';', decimal='.') 
df_gov = df_gov[df_gov['Country Code'].isin(target_iso_codes)]
cols_gov = ['Country Code'] + [str(y) for y in target_years_int]
existing_cols = [c for c in cols_gov if c in df_gov.columns]
df_gov = df_gov[existing_cols]
df_gov_melt = df_gov.melt(id_vars=['Country Code'], var_name='Year', value_name='Gov_Expenditure')
df_gov_melt['Year'] = df_gov_melt['Year'].astype(int)
df_gov_melt.rename(columns={'Country Code': 'Country_Code'}, inplace=True)
df_final = pd.merge(df_final, df_gov_melt, on=['Country_Code', 'Year'], how='left')

# --- 9. Employment Rate ---
empl_file = 'employment_rate.csv'
df_empl = pd.read_csv(empl_file, sep=';', decimal=',')
df_empl.rename(columns={df_empl.columns[0]: 'Country'}, inplace=True)
df_empl['Country_Code'] = df_empl['Country'].map(map_empl_to_iso)
df_empl = df_empl.dropna(subset=['Country_Code'])
df_empl = df_empl[df_empl['Country_Code'].isin(target_iso_codes)]
cols_empl = ['Country_Code'] + [str(y) for y in target_years_int]
existing_cols_empl = [c for c in cols_empl if c in df_empl.columns]
df_empl = df_empl[existing_cols_empl]
df_empl_melt = df_empl.melt(id_vars=['Country_Code'], var_name='Year', value_name='Employment_Rate')
df_empl_melt['Year'] = df_empl_melt['Year'].astype(int)
df_final = pd.merge(df_final, df_empl_melt, on=['Country_Code', 'Year'], how='left')

# --- 10. Public Health Expenditure ---
phe_file = 'public_health_expenditure_social_monitor_2004.csv'
df_phe = pd.read_csv(phe_file, sep=';', decimal='.')
df_phe['Country_Code'] = df_phe['Kraj'].map(map_pol_to_iso)
df_phe = df_phe.dropna(subset=['Country_Code'])
df_phe = df_phe[df_phe['Country_Code'].isin(target_iso_codes)]
cols_phe = ['Country_Code'] + [str(y) for y in target_years_int]
existing_cols_phe = [c for c in cols_phe if c in df_phe.columns]
df_phe = df_phe[existing_cols_phe]
df_phe_melt = df_phe.melt(id_vars=['Country_Code'], var_name='Year', value_name='Public_Health_Expenditure')
df_phe_melt['Year'] = df_phe_melt['Year'].astype(int)
df_final = pd.merge(df_final, df_phe_melt, on=['Country_Code', 'Year'], how='left')

# --- 11. Transition Reports Data (Private Sector, Inflation, Unemployment) ---
tr_file = 'transition_reports_data.csv'
df_tr = pd.read_csv(tr_file, sep=';', decimal=',', na_values='na')
df_tr['Country_Code'] = df_tr['Kraj'].map(map_pol_to_iso)
df_tr = df_tr.dropna(subset=['Country_Code'])
df_tr = df_tr[df_tr['Country_Code'].isin(target_iso_codes)]
df_tr = df_tr[['Country_Code', 'Rok', 'Sektor prywatny (% PKB)', 'Inflacja (%)', 'Stopa bezrobocia (%)']].copy()
df_tr.rename(columns={'Rok': 'Year', 'Sektor prywatny (% PKB)': 'Private_Sector_Share_GDP', 'Inflacja (%)': 'Inflation', 'Stopa bezrobocia (%)': 'Unemployment_Rate'}, inplace=True)
df_final = pd.merge(df_final, df_tr, on=['Country_Code', 'Year'], how='left')

# --- Finalize ---
df_final.insert(0, 'Kraj', df_final['Country_Code'].map(map_iso_to_pol))
df_final.sort_values(['Kraj', 'Year'], inplace=True)

# Clean decimals and quotes
for col in df_final.columns:
    if df_final[col].dtype == 'object' and col not in ['Kraj', 'Country_Code']:
        try:
            df_final[col] = df_final[col].astype(str).str.replace(',', '.', regex=False)
            df_final[col] = pd.to_numeric(df_final[col], errors='ignore')
        except:
            pass

# Save unquoted
output_filename = 'dane_zintegrowane_1989_2014.csv'
df_final.to_csv(output_filename, index=False, quoting=csv.QUOTE_NONE, escapechar='\\')

C:\Users\wojci\AppData\Local\Temp\ipykernel_5668\3490910664.py:190: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_final[col] = pd.to_numeric(df_final[col], errors='ignore')
